In [1]:
%pwd

'd:\\Projects\\chicken-disease-classification\\research'

Incase the pwd is not the home folder, change the directory to home folder
import os
os.chdir("../")

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\Projects\\chicken-disease-classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from pathlib import Path

class ConfiguratonManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            ):
        print("Configuration Manager Initiated")
        # print(f"Config File Path: {config_filepath}, with dtype {type(config_filepath)}")
        # print(f"Params File Path: {params_filepath} with dtype {type(params_filepath)}")
        self.configs = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # print(f"Configs: {self.configs}")
        create_directories([self.configs.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.configs.data_ingestion

        create_directories([config.root_dir])

        return DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

# config_dummy = ConfiguratonManager()


In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded file with the folowing headers: \n{headers}")
        
        else:
            logger.info(f"File already exists at {self.config.local_data_file} of size {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with ZipFile(self.config.local_data_file, 'r') as zip:
            zip.extractall(path=unzip_path)
            logger.info(f"Extracted file at {self.config.unzip_dir}")
                

In [11]:
try:
    config = ConfiguratonManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e
    

Configuration Manager Initiated
[ 2023-09-14 01:42:42,982: INFO: common:  yaml file: config\config.yaml loaded successfully]
[ 2023-09-14 01:42:42,987: INFO: common:  yaml file: params.yaml loaded successfully]
[ 2023-09-14 01:42:42,991: INFO: common:  directory: artifacts created successfully]
[ 2023-09-14 01:42:42,994: INFO: common:  directory: artifacts created successfully]
[ 2023-09-14 01:42:42,997: INFO: common:  directory: artifacts/data_ingestion created successfully]
[ 2023-09-14 01:42:43,000: INFO: common:  directory: artifacts/data_ingestion created successfully]
[ 2023-09-14 01:43:31,133: INFO: 653728985:  Downloaded file with the folowing headers: 
Connection: close
Content-Length: 24046553
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d78fcf292e31630cad7071fd7f1e4dc9b22a2822f11e105ce7c4c439da739169"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X